In [1]:
import sympy as sp
import numpy as np
import scipy
from scipy.linalg import solve_continuous_are, inv
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

## Question 1b

In [2]:
m1, I0, L0, g, l1, J1, t = sp.symbols('m1 I0 L0 g l1 J1 t') # constants
tau = sp.symbols('tau') # control
theta0, theta1 = dynamicsymbols('theta0 theta1')

In [3]:
theta0_dot, theta1_dot = sp.diff(theta0, t), sp.diff(theta1, t)

In [4]:
state = sp.Matrix([theta0, theta1, theta0_dot, theta1_dot])
control = sp.Matrix([tau])

In [5]:
q = sp.Matrix([theta0, theta1])
q_dot = sp.diff(q, t)
q_2dot = sp.diff(q_dot, t)

In [6]:
c11 = 0.5 * m1 * l1**2 * theta1_dot * sp.sin(2 * theta1)
c12 = -m1 * l1 * L0 * theta1_dot * sp.sin(theta1) + 0.5 * m1 * l1**2 * theta0_dot * sp.sin(2 * theta1)
c21 = -0.5 * m1 * l1**2 * theta0_dot * sp.sin(2 * theta1)
c22 = 0

In [7]:
C = sp.Matrix([[c11, c12], [c21, c22]])

In [8]:
m11 = I0 + m1 * (L0**2 + l1**2 * sp.sin(theta1) ** 2)
m12 = m1 * l1 * L0 * sp.cos(theta1)
m21 = m1 * l1 * L0 * sp.cos(theta1)
m22 = J1 + m1 * l1**2

In [9]:
M = sp.Matrix([[m11, m12], [m21, m22]])

In [10]:
g_q = sp.Matrix([0, -m1 * l1 * g * sp.sin(theta1)])

In [11]:
F = sp.Matrix([tau, 0])

In [12]:
dynamics = sp.diff(state, t)
theta0_2dot = dynamics[2]
theta1_2dot = dynamics[3]
dynamics

Matrix([
[     Derivative(theta0(t), t)],
[     Derivative(theta1(t), t)],
[Derivative(theta0(t), (t, 2))],
[Derivative(theta1(t), (t, 2))]])

In [13]:
eq = M * q_2dot + C * q_dot + g_q - F
eq

Matrix([
[L0*l1*m1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) + 0.5*l1**2*m1*sin(2*theta1(t))*Derivative(theta0(t), t)*Derivative(theta1(t), t) - tau + (I0 + m1*(L0**2 + l1**2*sin(theta1(t))**2))*Derivative(theta0(t), (t, 2)) + (-L0*l1*m1*sin(theta1(t))*Derivative(theta1(t), t) + 0.5*l1**2*m1*sin(2*theta1(t))*Derivative(theta0(t), t))*Derivative(theta1(t), t)],
[                                                                                                                                                                       L0*l1*m1*cos(theta1(t))*Derivative(theta0(t), (t, 2)) - g*l1*m1*sin(theta1(t)) - 0.5*l1**2*m1*sin(2*theta1(t))*Derivative(theta0(t), t)**2 + (J1 + l1**2*m1)*Derivative(theta1(t), (t, 2))]])

In [14]:
sol = sp.solve([eq[0], eq[1]], theta0_2dot, theta1_2dot)

In [15]:
theta0_2dot = sol[theta0_2dot]
theta1_2dot = sol[theta1_2dot]

In [16]:
dynamics[2] = theta0_2dot
dynamics[3] = theta1_2dot

In [17]:
A = dynamics.jacobian(state)
B = dynamics.jacobian(control)

In [18]:
A = A.subs([
            (g, 9.8),
            (theta0, 0),
            (theta1, 0),
            (theta0_dot, 0),
            (theta1_dot, 0),
            (m1, 1),
            (I0, 1),
            (L0, 1),
            (l1, 1),
            (J1, 1),
            (tau, 0),
           ]
          )
A.simplify()

In [19]:
B = B.subs([
            (g, 9.8),
            (theta0, 0),
            (theta1, 0),
            (theta0_dot, 0),
            (theta1_dot, 0),
            (m1, 1),
            (I0, 1),
            (L0, 1),
            (l1, 1),
            (J1, 1),
            (tau, 0),
           ]
          )
B.simplify()

In [20]:
A

Matrix([
[0,                 0, 1, 0],
[0,                 0, 0, 1],
[0, -3.26666666666667, 0, 0],
[0,  6.53333333333333, 0, 0]])

In [21]:
B

Matrix([
[                 0],
[                 0],
[ 0.666666666666667],
[-0.333333333333333]])

## Question 1c

In [22]:
def lqr(A: np.ndarray, B: np.ndarray, Q: np.ndarray, R: np.ndarray) -> np.ndarray:
    S = scipy.linalg.solve_continuous_are(A, B, Q, R)
    K = -scipy.linalg.inv(R).dot(B.T).dot(S)
    return K

In [23]:
A = np.array(A).astype(float)
B = np.array(B).astype(float)

In [24]:
Q = np.identity(4)
R = np.identity(1)

In [25]:
K_star = lqr(A, B, Q, R)
K_star

array([[ 1.        , 60.26600338,  3.0494564 , 24.92963293]])

In [26]:
def controller(state):
    # u_0 = 0
    # x_0 = 0
    return K_star @ state